# Transitioning nengo_spa coming from the core Nengo legacy SPA implementation

This tutorial is intended for persons who are already familiar with the legacy Semantic Pointer Architecture (SPA) implementation that was shipped with core Nengo until at least version 2.4. Thus, it will not explain any of the background behind the SPA and will reference concepts used in the legacy implementation. If you are completely new to the SPA, you might want to start with a different tutorial (TODO link it here once written).

## Why switch to nengo_spa?

You might wonder why you should switch to nengo_spa, if you have been using the legacy SPA and it was working well for you. Here is a number of reasons to prefer nengo_spa:

* Support for action rules of arbitrary complexity. Want to do `dot((role * filler + BiasVector) * tag, cmp) --> ...`? That is no problem with nengo_spa. Note that this includes two circular convolutions and a dot product of two non-fixed values; something not possible in this way with the legacy SPA implementation.
* “Type safety” in action rules. If different vocabularies are combined an explicit conversion is required which prevents hard-to-track-down bugs. This conversion also makes it explicit how the conversion is done instead of just applying a fixed method that is not always appropriate.
* The neural representations have been optimized to provide better accuracy. That means less neurons are needed to achieve the same performance which in turn can make simulations run faster. This improvement of accuracy is comparable to the results presented in [“Optimizing Semantic Pointer Representations for Symbol-Like Processing in Spiking Neural Networks”](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0149928) which achieved improvements of up to 25 times. In contrast to that paper, the implementation used in nengo_spa is purely based on setting appropriate radii and distributions for evaluation points as well as intercepts which makes it a lot faster.
* The neural representation has been optimized to allow the representation of the identity vector (with the option to turn this optimization off).
* SPA networks can be used as and within normal Nengo networks. For example, instead of a basic and an SPA associative memory network, there is only one type of network that can be used in either case.
* SPA networks can be nested.
* Support for the Nengo config system.
* Lots of fixed issues and in general less possibilities to do things wrong.

## Importing nengo_spa

To save typing it is recommended to `import nengo_spa as spa`. Note that this is only a very small difference to `import nengo.spa as spa` which imports the legacy SPA. In the following, it is assumed that you imported `nengo_spa` as `spa`.

In [1]:
%matplotlib inline
%load_ext nengo.ipynb

In [2]:
import matplotlib.pyplot as plt
import nengo
import nengo_spa as spa
import numpy as np

In [3]:
d = 32  # Default dimensionality to use in the examples

## Rule 1 of using nengo_spa

When using `nengo_spa`, use `spa.Network()` in any place where you would use a `nengo.Network()`. There isn't any downside to using `spa.Network()` and in this way you will not run into problems of using `nengo.Network()` where `spa.Network()` is required. That being said, if you have existing `nengo.Network`s (that do not use any SPA features), these can stay `nengo.Network`s. In other words, you will only have to touch code using the SPA to upgrade to nengo_spa; networks not using the SPA system may stay as they are.

If you want to know when exactly `spa.Network()` is required, here are the conditions:

1. If you build action rules, they have to be build in a `spa.Network()`.
2. Each network referenced in an action rule has to be a  `spa.Network()` (if accessing `model.level1.level2`, all three `model`, `level1`, and `level2` need to be `spa.Network()`).

# A basic model

In [4]:
with spa.Network() as model:
    model.state = spa.State(d)
    model.bind = spa.Bind(d)
    spa.Actions('state = A * B', 'bind.input_a = state', 'bind.input_b = ~B').build()
    p = nengo.Probe(model.bind.output, synapse=0.03)

This defines a basic SPA model. It is pretty similar to what you were to write in legacy SPA. But there are some fine details, so let us go through each of those.

The first difference is that instead of `with spa.SPA() ...`, we are using `with spa.Network() ...`. There is no difference between the top-level `spa.SPA` and `spa.Module` anymore. Everything is just a `spa.Network`.

The next two lines are the same as for the legacy SPA. You define the modules with the vocabulary dimension and assign them as attributes to the model. Note that you do not have to assign those modules as attributes as long as you are not trying to access them in action rules.

The `spa.Actions` line is what replaces `nengo.spa.Cortical`. The cortical action rules are written in essentially the same way, though to access a specific input the typical Python dot notation is used instead of an underscore. Also note that the names of some of the inputs have changed. They follow a consistent naming scheme now: always starting with `input_` and you use the same name in action rules as in manually created Nengo connections. Finally, to create those cortical connections, you do not use a `Cortical` object, but the `build()` function on the `Actions` object which will build the connections within the currently active network. Note that the names in the actions rules are also relative to the currently active network and need to refer to assigned `spa.Network` attributes.

In [5]:
with nengo.Simulator(model) as sim:
    sim.run(1.)

Let us plot the result. Note that we can access the *d*-dimensional default vocabulary with `model.vocabs[d]`.

In [6]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.vocabs[d]))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(model.vocabs[d].keys(), loc='best')

There is another detail you might notice here: the `keys` attribute on the vocabularies is now a function. This is to have `Vocabulary` adhere to the usual Python API of dictionaries and mapping types.

## Encode, Decode, Transcode

In the previous example, input was provided directly in the action rules. But this does not allow for time-varying inputs. The legacy SPA `Input` has been replaced with `spa.Encode` which has an API more similar to classic `nengo.Nodes`. It takes either a constant value or a function which may return any of the following:

* A string that is parsed as a semantic pointer.
* A `SemanticPointer` instance.
* A NumPy array.

The next examples demonstrates this. It also manually specifies a vocabulary, but ignore this for now as the changes to `Vocabulay` will be discussed later.

In [7]:
vocab = spa.Vocabulary(d, strict=False)

def stimulus_fn(t):
    if t < 0.5:
        return 'A * B'  # Return string to be parsed
    elif t < 1.:
        return vocab.parse('C * B')  # Return SemanticPointer instance
    else:
        return np.zeros(d)  # Return a numpy array

with spa.Network() as model:
    model.state = spa.State(vocab)
    model.bind = spa.Bind(vocab)
    model.input = spa.Encode(stimulus_fn, vocab)
    spa.Actions('state = input', 'bind.input_a = state', 'bind.input_b = ~B').build()
    p = nengo.Probe(model.bind.output, synapse=0.03)

In [8]:
with nengo.Simulator(model) as sim:
    sim.run(1.5)

In [9]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], vocab))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(vocab.keys(), loc='best')

One thing that is possible with `nengo.Node`, but not with `spa.Encode` (nor with the old `spa.Input`), is to provide an input from the network. If you want to do this, you can now use the `spa.Transcode` module. Its output function gets three inputs: the current simulation time, a `SemanticPointer` instance of the input, and the input vocabulary. To create a `Transcode` instance, the dimensionality (or vocabulary) has to be passed in twice. The first specifies the input vocabulary (the one passed to the output function), the second one specifies the output vocabulary that will be used to pares the function's output.

In the following example we compute the circular convolution with `~B` in math instead of a neural network.

In [10]:
def cconv(t, p, vocab):
    return vocab.parse('~B') * p

with spa.Network() as model:
    model.state = spa.State(d)
    model.bind = spa.Transcode(cconv, d, d)
    model.input = spa.Encode('A * B', d)
    spa.Actions('state = input', 'bind = state').build()
    p = nengo.Probe(model.bind.output, synapse=0.03)

In [11]:
with nengo.Simulator(model) as sim:
    sim.run(1.)

In [12]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.vocabs[d]))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(model.vocabs[d].keys(), loc='best')

Besides `Encode` and `Transcode`,  there is a third module called `Decode`. It accepts an input like with same function arguments as for `Transcode`, but the function cannot provide any input to the network.

As a recap: `Encode` encodes Semantic Pointers into neural activity, `Decode` decodes Semantic Pointers from neural activity, and `Transcode` decodes Semantic Pointers from neural activity and reencodes them after some transformation.

## Action rules

We have already seen how to create cortical action rules with nengo_spa. The next example shows how to add action rules implemented through the basal ganglia-thalamus loop. It is the classical routing through a sequence example.

In [13]:
def start(t):
    if t < 0.05:
        return 'A'
    else:
        return '0'

with spa.Network() as model:
    model.state = spa.State(d)
    model.input = spa.Encode(start, d)
    
    spa.Actions(
        'state = input',
        'dot(state, A) --> state = B',
        'dot(state, B) --> state = C',
        'dot(state, C) --> state = D',
        'dot(state, D) --> state = E',
        'dot(state, E) --> state = A'
    ).build()
    
    p = nengo.Probe(model.state.output, synapse=0.01)

In [14]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [15]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.vocabs[d]))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")

Note that you can freely mix cortical and basal ganglia rules. There is also no need to manually create the basal ganglia and thalamus. This happens automatically. If you build multiple `Actions` objects, multiple instances will be created that operate in parallel. So in most cases you want to collect all you rules in a single `Actions` object. In case you need access to the created basal ganglia and thalamus (e.g. for plotting as in the next example), those objects are returned from the `build` function.

In [16]:
def start(t):
    if t < 0.05:
        return 'A'
    else:
        return '0'

with spa.Network() as model:
    model.state = spa.State(d)
    model.input = spa.Encode(start, d)
    
    actions = spa.Actions(
        'state = input',
        'dot(state, A) --> state = B',
        'dot(state, B) --> state = C',
        'dot(state, C) --> state = D',
        'dot(state, D) --> state = E',
        'dot(state, E) --> state = A'
    )
    model.bg, model.thalamus, _ = actions.build()
    
    p_thalamus = nengo.Probe(model.thalamus.actions.output, synapse=0.01)
    p_utility = nengo.Probe(model.bg.input, synapse=0.01)

In [17]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [18]:
plt.subplot(2, 1, 1)
plt.plot(sim.trange(), sim.data[p_thalamus])
plt.legend([a.effects for a in actions], fontsize='x-small')
plt.ylabel('Thalamus output')

plt.subplot(2, 1, 2)
plt.plot(sim.trange(), sim.data[p_utility])
plt.legend([getattr(a, 'condition', '<cortical>') for a in actions], fontsize='x-small', loc='right')
plt.ylabel('Utility')

Opposed to the legacy SPA, nengo_spa allows arbitrarily complex actions rules. In the following example we define dot products between two states and dynamic circular convolutions. All required networks will be created automatically.

In [19]:
with spa.Network() as model:
    model.state_ab = spa.State(d)
    model.state_a = spa.State(d)
    model.state_b = spa.State(d)
    model.out = spa.State(d)
    
    actions = spa.Actions(
        'dot(state_ab, state_a * state_b) --> out = state_ab * ~B',
        '0.5 --> out = C',
        'state_ab = A * B',
        'state_a = A',
        'state_b = state_ab * ~state_a',
    )
    model.bg, model.thalamus, constructed = actions.build()
    
    p = nengo.Probe(model.out.output, synapse=0.01)

In [20]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [21]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.vocabs[d]))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(model.vocabs[d].keys())

Note that there is no automatic sharing of networks in the action rules. If, for example, you define the same circular convolution in there, two such networks will be created.

If you have nengo_gui installed, it can be used to take a look at all the things that get created.

In [22]:
try:
    from nengo_gui.ipython import IPythonViz
    vis = IPythonViz(model)
except (ImportError, AttributeError):
    print("GUI not installed or code not executed in Jupyter notebook.")
    vis = None
vis

Another way to access all the created objects are the three values returned by `build()`. The first two are the basal ganglia and thalamus as already discussed and give access to all related networks and connections. All the remaining objects that were created are provided in a dictionary `constructed` returned as third value. The dictionary uses nodes in the internally constructed syntax tree to index lists of objects created for that node (these nodes are not related to `nengo.Node`!). Admittedly, this is not the most convenient way to access things and might change in the future. But for now it at least gives a possibility to access those objects if required.

In [23]:
from pprint import pprint
pprint(constructed)

## Vocabularies

Nengo_spa behaves a lot like legacy SPA when providing dimensionalities only. Vocabularies will implicitely be created according to the specified dimensions and Semantic Pointers will be automatically added. When creating a vocabulary explicitely, things are a little bit different. In that case the vocabulary will be in strict-mode by default. That means an exception will be raised when trying to parse a Semantic Pointer that is not in the vocabulary. This is to prevent accidentally adding new Semantic Pointers (something that tendend to happen with the associative memmory in legacy SPA) and can make it easier to notice typing errors.

In [27]:
from nengo_spa.exceptions import SpaParseError
vocab = spa.Vocabulary(d)
try:
    vocab.parse('A')
except SpaParseError as err:
    print(err)

All Semantic Pointers have to be added to the vocabulary with either `add` or the new `populate` method before they are recognized by `parse`. You can add multiple pointers add once by separating them with a semicolon in `populate`.

In [28]:
vocab = spa.Vocabulary(d)
vocab.add('A', vocab.create_pointer())
vocab.populate('B; C')
vocab.parse('A + B + C')

If you prefer to automatically add unknown vectors, you can disable strict mode. This can be especially useful when experimenting with initial ideas in the GUI.

In [29]:
vocab = spa.Vocabulary(d, strict=False)
vocab.parse('A')

The new `populate` method is much more powerful than adding pointers with `parse`. For example you can use existing Semantic pointers to construct new ones and you can use transforms as `normalized()` and `unitary()` to make vectors unit length or normalized. Note that a simple Semantic Pointer will be normalized, but you need to explicitely do this when constructing a pointer out of others.

In the following example we create a vocabulary with four pointers *A*, *B*, *C*, and *D*. *A* is made unitary, D is constructed from other vectors and normalized.

In [30]:
vocab = spa.Vocabulary(d)
vocab.populate('A.unitary(); B; C; D = (A * B + C).normalized()')

Another new and sometimes useful method is `parse_n` which allows to parse multiple Semantic Pointer expressions at once. This can be useful for programatically constructing a list of pointers for plotting. The following example demonstrates that for all convolution pairs. It also shows that when using a predefined vocabulary, the modules will obtain their dimensionality from that vocabulary. No need to pass dimensionality and vocabulary anymore!

In [31]:
vocab = spa.Vocabulary(d)
vocab.populate('A; B; C')

with spa.Network() as model:
    model.state = spa.State(vocab)
    spa.Actions('state = A * B').build()
    p = nengo.Probe(model.state.output, synapse=0.01)

In [32]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [33]:
from nengo_spa.examine import pairs
plot_keys = pairs(vocab)
plot_vectors = vocab.parse_n(*plot_keys)

plt.plot(sim.trange(), spa.similarity(sim.data[p], plot_vectors))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(plot_keys)

Another feature of nengo_spa is “type-safety” in the sense that you cannot just connect things with different vocabularies in action rules.

In [36]:
from nengo_spa.exceptions import SpaTypeError

v1 = spa.Vocabulary(d)
v1.populate('A; B')
v2 = spa.Vocabulary(d)
v2.populate('B; C')

try:
    with spa.Network() as model:
        model.state1 = spa.State(v1)
        model.state2 = spa.State(v2)
        spa.Actions('state1 = state2').build()
except SpaTypeError as err:
    print(err)

If we want to do this, we have to be explicit about how the conversion between the vocabularies is supposed to be happen. The first option (if both vocabularies have the same dimensionality) is to just reinterpret the Semantic Pointer in the other vocabulary. Because the vectors in both vocabularies are independent (by default) the *A* from `v2` will be different from *A* in `v1`.

In [37]:
v1 = spa.Vocabulary(d)
v1.populate('A; B')
v2 = spa.Vocabulary(d)
v2.populate('A; B')

with spa.Network() as model:
    model.state1 = spa.State(v1)
    model.state2 = spa.State(v2)
    spa.Actions('state1 = reinterpret(state2)', 'state2 = A').build()
    p = nengo.Probe(model.state1.output, synapse=0.01)

In [38]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [39]:
plt.plot(sim.trange(), v1['A'].dot(sim.data[p].T))
plt.plot(sim.trange(), v2['A'].dot(sim.data[p].T))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(["v1['A']", "v2['A']"])

We see that the value in `state2` is still similar to `v2`'s *A* even though `state2` uses `v1` as vocabulary.

The second choice to convert between vocabularies is `translate` which will construct a transformation matrix to convert from one vocabulary to the other based on the Semantic Pointer names. This also works with vocabularies that do not match in dimensionality, but the target vocabulary should contain all keys of the source vocabulary. If this is not the case, you will get either a warning or an exception depending on whether you are using strict-mode vocabularies. You can also use the `populate=True` argument to `translate` to have all missing keys added to the target vocabulary.

In [40]:
v1 = spa.Vocabulary(d)
v1.populate('A; B')
v2 = spa.Vocabulary(d)
v2.populate('A; B')

with spa.Network() as model:
    model.state1 = spa.State(v1)
    model.state2 = spa.State(v2)
    spa.Actions('state1 = translate(state2)', 'state2 = A').build()
    p = nengo.Probe(model.state1.output, synapse=0.01)

In [41]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [42]:
plt.plot(sim.trange(), v1['A'].dot(sim.data[p].T))
plt.plot(sim.trange(), v2['A'].dot(sim.data[p].T))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(["v1['A']", "v2['A']"])

## nengo_spa and the config system

One of the major improvements in nengo_spa is the extensive use of Nengo's config system. For example it allows to set the vocab for all states globally.

In [43]:
with spa.Network() as model:
    model.config[spa.State].vocab = d
    model.state1 = spa.State()
    model.state2 = spa.State()
    spa.Actions('state2 = state1', 'state1 = A').build()
    p = nengo.Probe(model.state2.output, synapse=0.01)

In [44]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [45]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.vocabs[d]))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(model.vocabs[d].keys())

The config system can also be used to set neuron numbers, subdimensions, and various other parameters, in particular of objects created when building action rules.

In [46]:
def start(t):
    if t < 0.1:
        return 'A'
    else:
        return '0'

with spa.Network() as model:
    model.config[spa.State].neurons_per_dimension = 10
    model.config[spa.State].subdimensions = 1
    model.config[spa.Thalamus].synapse_channel = nengo.Lowpass(0.1)
    
    model.state = spa.State(d)
    model.input = spa.Encode(start, d)
    spa.Actions(
        'state = input',
        'dot(state, A) --> state = 2 * B',
        'dot(state, B) --> state = 2 * C',
        'dot(state, C) --> state = 2 * A',
    ).build()

    p = nengo.Probe(model.state.output, synapse=0.01)

In [47]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

In [48]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.vocabs[d]))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(model.vocabs[d].keys())

## Semantic Pointers

The `SemanticPointer` class has become immutable in nengo_spa to avoid accidental modifiction of Semantic Pointers. Whenever an operation is applied to a `SemanticPointer` (for example `SemanticPointer(d) + SemanticPointer(d)` or `SemanticPointer(d).normalized()` a new instance will be created.

To facilitate mathematical analysis of encoding schemes some special vectors are predefined:

* `nengo_spa.pointer.Identity(d)` gives the identity vector for circular convolution.
* `nengo_spa.pointer.Zero(d)` gives the vector of all zeros (absorbing element for circular convolution).
* `nengo_spa.pointer.AbsorbingElement(d)` gives a vector that destroys all information under circluar convolution except for a DC offset.

## Representing identity

In nengo_spa `spa.State()` is optimized to allow the representation of the identity vector. This is not always necessary as in many models the identity never needs to represented. With `represent_identity=False` this optimization can be disabled. This can make the representation for non-identity vectors slightly better. It also simplifies the internal structure of `spa.State()` which can be helpful for applying learning rules.

## New associative memory classes

To reduce the number of arguments of the associative memory class it has been split into two classes. `spa.ThresholdingAssocMem` is the applying a simple threshold, `spa.WTAAssocMem` performs a winner-take-all clean-up with lateral inhibitory connections. There is also a new type of associative memory `spa.IAAssocMem` based on independent accumulators, that also exhibits winner-take-all behaviour, but with different dynamics. (TODO link CogSci paper once published)

To implement auto-associative memories, the `input_keys` and `output_keys` arguments have been replaced by a single `mapping` arguments.

TODO link associative memory documentation/tutorial.